# Refined Power Analysis for RE

This notebook explores using the RPA attack for reverse-engineering.
You will first explore scalar multipliers and the multiples they compute.
Then, you will study the RPA attack.
Finally you will put the knowledge to use in reverse-engineering.

In [28]:
from pyecsca.ec.params import get_params, DomainParameters
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mod import Mod
from pyecsca.ec.point import Point
from pyecsca.ec.mult import ScalarMultiplier

### TODO
 - Multipliers differ in their "intermediate scalars" that they compute while computing a scalarmult.
     - How to explore this?
     - LTR vs RTL? prefixes vs suffixes vs windows?
 - In the RPA attack, which attacks static ECDH, one is able to detect a particular multiple of the input point is computed, by crafting a special input point and using a power side-channel.
     - How to explore this?
     - It is an "oracle", you are able to ask questions like "is $[k]P$ computed in a given implementation when it is doing scalarmult of $[r]P$?"
     - Show the RPA attack. Existence of $P_0$ point, then, when you do $[k^{-1}]P_0 = P$ you get a point such that $[k]P = [k][k^{-1}]P_0 = P_0$. Then get two groups of traces of the implementation computing scalarmult with a secret (fixed) scalar, one group with random input points, other with point $P$ for some $k$. If $[k]P$ is computed, you will see a difference between these two groups at some point, because of the zero coordinate. To spot the difference, basically do DPA: average the groups and subtract their average traces, then see spike (peak detect or threshold).
 - Putting these two things together, we see that we can build a reverse-engineering method out of them.

## Scalar multipliers
Scalar multipliers come in all sorts of different shapes, sizes and colors. 🌈

In [ ]:
from pyecsca.ec.mult import LTRMultiplier, RTLMultiplier
from pyecsca.sca.re.rpa import MultipleContext

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Compare the chain of multiples that is computed in the left-to-right vs right-to-left scalar multiplier for a given fixed scalar.
- What do these intermediate multiples (scalars) represent in the two cases?
- When are the chains the same? When are the sets of multiples the same?

**Docs**<br/>
[LTRMultiplier](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mult.binary.html#pyecsca.ec.mult.binary.LTRMultiplier)<br/>
[RTLMultiplier](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mult.binary.html#pyecsca.ec.mult.binary.RTLMultiplier)<br/>
[MultipleContext](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.MultipleContext)

In [ ]:
params = get_params("secg", "secp256r1", "projective")
add = params.curve.coordinate_model.formulas["add-2007-bl"] # The formulas do not matter, but we need some
dbl = params.curve.coordinate_model.formulas["dbl-2007-bl"] # The formulas do not matter, but we need some

ltr_mult = LTRMultiplier(add, dbl)
rtl_mult = RTLMultiplier(add, dbl)

# Task: Fix a scalar
k = ...
P = params.generator

with local(MultipleContext()) as ltr_ctx:
    # Task: Init and multiply using the LTR multiplier here

with local(MultipleContext()) as rtl_ctx:
    # Task: Init and multiply using the RTL multiplier here 

# Task: Compare the ltr_ctx and rtl_ctx.
# You can look at their .points and .parents attributes.

## RPA attack
The RPA attack (by Louis Goubin, [A Refined Power-Analysis Attack on Elliptic Curve Cryptosystems](http://www.goubin.fr/papers/ecc-dpa.pdf)) works by introducing a zero-coordinate-point into the scalar multiplication conditionally on a secret key part and then detecting the presence of the zero via a side-channel. The target is static ECDH, in which case there is a static scalar $k$ and the target is willing to compute $[k]P$ for any (reasonable) point $P$.

### Smuggling the zero in

In the attack, the existence of zero-coordinate points is important.
They are either of the form $P_0 = (x, 0)$ or $P_0 = (0, y)$. The first form is less interesting, as it represents points of order 2, which are not present on prime-order elliptic curves often used in practice.

During the attack, the attacker can introduces these zero-coordinate points into the scalar multiplication conditionally on some secret key hypothesis. Upon input of $P = [r^{-1}]P_0$ into the scalar multiplier, the point $P_0$ will appear inside iff the multiplier computes the $r$-th multiple of the input point, as $[r]P = [r][r^{-1}]P_0 = P_0$. In the attack, the attacker uses this to confirm hypotheses about parts of the secret key.

### Detecting the zero

Now that we have a way of getting the zero in, we need a way of detecting it. We will use a power (or EM) side-channel and the fact that arithmetic operations with zero values will leak in a different way than operations with non-zero values. The diagram below shows the idea (which is very similar to DPA). We will collect two groups of traces, one with the special point $P=[r^{-1}]P_0$ and one with random points $P$ input into the target. We will then average the traces in the two groups and subtract the average traces to form a Difference-of-Means (DoM) trace. If this traces has a peak we were able to distinguish between the two groups, which implies that the zero did occur in the "special point $P$" group, which in turn implies that the scalar multiplier computed the $r$-th multiple of the input point.

![](img/rpa.svg)

### Nuances

The RPA attack will not work in the presence of scalar randomization countermeasures, as these effectively randomize the sequence of multiples computed while computing the $[k]P$. However, the attack is resistant against coordinate and curve randomization, because the zero-coordinate points resist randomization.

In [29]:
from pyecsca.sca.re.rpa import rpa_point_0y, rpa_point_x0
from pyecsca.sca.trace import Trace
from pyecsca.sca.attack.leakage_model import HammingWeight

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Practice the elements of the RPA attack.
 - Create a point P, such that $P_0$ occurs in the scalar multiplication $[k]P$ using the LTR multiplier but not the RTL multiplier.
 - Verify your construction of the point.
 - Construct the RPA oracle "box" from the figure above, compare it to a simulated one.

**Docs**<br/>
[rpa_point_0y](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.rpa_point_0y)<br/>
[Mod](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mod.html#pyecsca.ec.mod.Mod)<br/>
[affine_multiply](https://neuromancer.sk/pyecsca/api/pyecsca.ec.curve.html#pyecsca.ec.curve.EllipticCurve.affine_multiply)<br/>
[MultipleContext](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.MultipleContext)<br/>
[combine module (average, subtract, ...)](https://neuromancer.sk/pyecsca/api/pyecsca.sca.trace.combine.html)<br/>

In [ ]:
P0 = rpa_point_0y(params)
print(P0)
# Task: Create a point P, such that P0 occurs in the scalar multiplication of [k]P using the LTR multiplier
#       but not the RTL multiplier.
# Hint: You can use params.curve.affine_multiply.

In [ ]:
# Task: Verify that your construction of the point is correct by simulating the multipliers using the MultipleContext
#       as above and verifying that the point is present when it should be.

In [30]:
def simulate_trace(mult: ScalarMultiplier, point: Point, scalar: int, params: DomainParameters) -> Trace:
    with local(DefaultContext()) as ctx:
        mult.init(params, point)
        mult.multiply(scalar)

    lm = HammingWeight()
    trace = []

    def callback(action):
        if isinstance(action, FormulaAction):
            for intermediate in action.op_results:
                leak = lm(intermediate.value)
                trace.append(leak)

    ctx.actions.walk(callback)
    return Trace(np.array(trace))

def simulated_oracle(mult: ScalarMultiplier, point: Point, scalar: int, params: DomainParameters) -> bool:
    with local(MultipleContext()) as ctx:
        mult.init(params, point)
        mult.multiply(scalar)
    points = set(ctx.parents.keys())
    parents = set(sum((ctx.parents[point] for point in points), []))
    zero = any(map(lambda P: P.X == 0 or P.Y == 0, parents)) # Did zero happen in some input point during the scalarmult?
    return zero

def rpa_oracle(mult: ScalarMultiplier, point: Point, scalar: int, params: DomainParameters) -> bool:
    # simulate a group of traces using "point" (this is P = [r^{-1}]P_0 from the diagram)
    # simulate a group of traces using random points
    # average, subtract, and decide on the result
    pass

# Task: Use the "simulate_trace" function to implement the "rpa_oracle" function
# Task: Compare your "rpa_oracle" function to the "simulated_oracle" function that is the ground truth

TODO: Maybe make the "nuances" part a question/exercise actually?

## Putting it together


TODO: Task for actually reverse-engineering.

Pick a scalar, collect answers from simulated oracle into table.
Then, how do you proceed to most effectively distinguish configs? Which input/question to pick?
-> distinguishing tree

TODO: Task, show that some countermeasures do not work (coordinate randomization).

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

**Docs**